Import Libraries

In [17]:
import pandas as pd
import sqlite3
import os

Create Folder Structure

In [18]:
os.makedirs("raw", exist_ok=True)
os.makedirs("processed", exist_ok=True)
os.makedirs("output", exist_ok=True)

Load CSV Files

In [19]:
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")

print("Loaded successfully")

Loaded successfully


Cleaning (Missing Values + Duplicates)

In [20]:
orders = orders.drop_duplicates()
order_items = order_items.drop_duplicates()
products = products.drop_duplicates()
customers = customers.drop_duplicates()

products = products.fillna("Unknown")
customers = customers.dropna(subset=["customer_id"])
orders = orders.dropna(subset=["order_id"])

Standardize Columns + Datatypes

In [21]:
orders.columns = orders.columns.str.lower()
order_items.columns = order_items.columns.str.lower()
products.columns = products.columns.str.lower()
customers.columns = customers.columns.str.lower()

orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp']
)

Merge + Derived Columns

In [22]:
df = order_items.merge(orders, on="order_id", how="left")

df["margin"] = df["price"] - df["freight_value"]
df["high_value_flag"] = df["price"].apply(lambda x: 1 if x > 100 else 0)

Split Outputs

In [23]:
customers_output = customers[
    ["customer_id","customer_unique_id","customer_city","customer_state"]
]

products_output = products[
    ["product_id","product_category_name"]
]

orders_output = df[
    ["order_id","product_id","price","freight_value","margin","high_value_flag"]
]

Export CSV Deliverables

In [24]:
df.to_csv("processed/cleaned_master_dataset.csv", index=False)

In [25]:
customers_output.to_csv("output/customers.csv", index=False)
products_output.to_csv("output/products.csv", index=False)
orders_output.to_csv("output/orders.csv", index=False)

Load into SQLite

In [26]:
conn = sqlite3.connect("database.sqlite")

customers_output.to_sql("customers", conn, if_exists="replace", index=False)
products_output.to_sql("products", conn, if_exists="replace", index=False)
orders_output.to_sql("orders", conn, if_exists="replace", index=False)

conn.close()

print("SQLite database created successfully")

SQLite database created successfully


Validate Counts

In [27]:
print("Original Rows:", len(order_items))
print("Processed Rows:", len(df))

print("Customers:", len(customers_output))
print("Products:", len(products_output))
print("Orders:", len(orders_output))

Original Rows: 112650
Processed Rows: 112650
Customers: 99441
Products: 32951
Orders: 112650
